In [1]:
import requests
import lxml.html as lh
import pandas as pd

In [2]:
url = 'https://tools.morningstar.co.uk/uk/stockreport/default.aspx?tab=10&vw=is&SecurityToken=0P000003MC%5D3%5D0%5DE0WWE%24%24ALL&Id=0P000003MC&ClientFund=0&CurrencyId=BAS'


In [3]:
#Create a handle, page, to handle the contents of the website
page = requests.get(url)


In [4]:
#Store contents of the website under doc
doc = lh.fromstring(page.content)


In [5]:
#Parse data stored between <tr>
tr_elements = doc.xpath('//tr')
tr_elements.pop(0)

<Element tr at 0x1f7abc5b868>

In [6]:
#Check length of first 12 rows
[len(T) for T in tr_elements[:20]]

[6, 6, 6, 6, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 1, 6]

In [7]:
#Parse first row as header (financial years)
#Create empty list
col=[]
i=0

#For each row, store each first elements (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name = t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))

1:""
2:"2015"
3:"2016"
4:"2017"
5:"2018"
6:"2019"


In [8]:
#Since first row is header, data stored on second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #if row is not of size 6, the //tr data is not from our table (should just delete primary headings but retain line items)
    if len(T)!=6:
        continue
        
    #i is the index of our column
    i=0
    
    #iterate through each element of row
    for t in T.iterchildren():
        data = t.text_content()
        #check if row is empty
        if i>0:
            try:
                data=int(data)
            except:
                pass
        #Append data to empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1
        

In [9]:
#check column lengths
[len(C) for (title,C) in col]

[18, 18, 18, 18, 18, 18]

In [10]:
#Create DataFrame
Dict = {title:column for (title,column) in col}
df = pd.DataFrame(Dict)


,,2015,2016,2017,2018,2019
0,Revenue,"16,192.00","12,399.00","20,322.00","30,391.00","23,406.00"
1,Cost of Revenue,"10,977.00","9,894.00","11,886.00","12,500.00","12,704.00"
2,Gross Operating Profit,"5,215.00","2,505.00","8,436.00","17,891.00","10,702.00"
3,Research and development,"1,540.00","1,617.00","1,824.00","2,141.00","2,441.00"
4,"Sales, general and administrative",719.00,659.00,743.00,813.00,836.00
